In [1]:
from __future__ import (print_function, absolute_import)
from __future__ import (division, unicode_literals)

# importing

import numpy as np
from astropy.io import fits
from astropy import units
from glob import glob
import os
import argparse

from pypeit import ginga
from pypeit.core import pixels
from pypeit import traceslits
from pypeit import processimages
from pypeit import scienceimage
from pypeit import arcimage
from pypeit.core import arc
from pypeit import wavecalib
from pypeit import wavetilts
from pypeit import waveimage
from pypeit import flatfield
from pypeit import traceimage
from pypeit import specobjs
from pypeit import utils
from pypeit import biasframe

# Spectrgraph and Settings
from pypeit.spectrographs.util import load_spectrograph
from pypeit.par import pypeitpar
from pypeit import ginga
from pypeit.core.skysub import global_skysub
from pypeit.core import skysub
from pypeit.core import extract
from pypeit.core.flux import generate_sensfunc
#from pypeit.core.flux import generate_sensfunc
from linetools import utils as ltu

from IPython import embed
from matplotlib import pyplot as plt

In [2]:
# import os
# os.system('ginga --modules=RC&')

In [3]:
spectro_name = 'vlt_xshooter_vis'
spectrograph = load_spectrograph(spectrograph=spectro_name)

# Detector settings
par = spectrograph.default_pypeit_par()

print(spectrograph.detector)

[Parameter      Value           Default  Type              Callable
------------------------------------------------------------------
dataext        0               0        int               False   
specaxis       0               0        int               False   
specflip       False           False    bool              False   
spatflip       False           False    bool              False   
xgap           0.0             0.0      int, float        False   
ygap           0.0             0.0      int, float        False   
ysize          1.0             1.0      int, float        False   
platescale     0.16            0.135    int, float        False   
darkcurr       0.0             0.0      int, float        False   
saturation     65535.0         65535.0  int, float        False   
mincounts      -1000.0         -1000.0  int, float        False   
nonlinear      0.86            0.86     int, float        False   
numamplifiers  1               1        int               Fal

In [9]:
xs_dir = '/home/ema/Dropbox/XshooterTestBed/VLT_XSHOOTER/'

# binning = '1x1'
# binning = '1x2'
binning = '2x2'

if binning is '1x1':
    print('1x1 binning')
    xsvis1x1_dir = xs_dir+'VIS_1x1/'

    bias_files = [xsvis1x1_dir+'XSHOO.2010-04-28T10:23:42.901.fits.gz',
                  xsvis1x1_dir+'XSHOO.2010-04-28T10:26:26.465.fits.gz',
                  xsvis1x1_dir+'XSHOO.2010-04-28T10:29:10.029.fits.gz']
    flat_files = [xsvis1x1_dir+'XSHOO.2010-04-28T12:24:28.466.fits.gz',
                  xsvis1x1_dir+'XSHOO.2010-04-28T12:25:39.192.fits.gz',
                  xsvis1x1_dir+'XSHOO.2010-04-28T12:26:49.168.fits.gz']

if binning is '1x2':
    print('1x2 binning')
    xsvis1x2_dir = xs_dir+'VIS_1x2/'

    bias_files = [xsvis1x2_dir+'XSHOO.2016-08-02T10:45:45.410.fits.gz',
                  xsvis1x2_dir+'XSHOO.2016-08-02T10:47:16.857.fits.gz',
                  xsvis1x2_dir+'XSHOO.2016-08-02T10:48:48.184.fits.gz']

    flat_files = [xsvis1x2_dir+'XSHOO.2016-08-02T13:36:24.258.fits.gz',
                  xsvis1x2_dir+'XSHOO.2016-08-02T13:37:24.132.fits.gz',
                  xsvis1x2_dir+'XSHOO.2016-08-02T13:38:24.917.fits.gz']

if binning is '2x2':
    print('2x2 binning')
    xsvis2x2_dir = xs_dir+'VIS_2x2/'

    bias_files = [xsvis2x2_dir+'XSHOO.2016-10-08T10:07:38.147.fits.gz',
                  xsvis2x2_dir+'XSHOO.2016-10-08T10:08:35.051.fits.gz',
                  xsvis2x2_dir+'XSHOO.2016-10-08T10:09:30.026.fits.gz']

    flat_files = [xsvis2x2_dir+'XSHOO.2016-10-08T13:03:20.598.fits.gz',
                  xsvis2x2_dir+'XSHOO.2016-10-08T13:03:57.921.fits.gz',
                  xsvis2x2_dir+'XSHOO.2016-10-08T13:04:35.024.fits.gz']

2x2 binning


In [10]:
par['calibrations']['biasframe']['number'] = 3

bImage = biasframe.BiasFrame(spectrograph, files=bias_files)

bimage = bImage.process()

[WARNING] :: processimages.py 541 process() - Your images have not been bias subtracted!
[INFO]    :: combine.py 62 comb_frames() - Combining 3 bias frames
[WORK IN ]::
[PROGRESS]:: combine.py 66 comb_frames() - lscomb feature has not been included here yet...
[INFO]    :: combine.py 95 comb_frames() - Finding saturated and non-linear pixels
[INFO]    :: combine.py 119 comb_frames() - Rejecting cosmic rays
[INFO]    :: combine.py 163 comb_frames() - Not rejecting any low/high pixels
[INFO]    :: combine.py 171 comb_frames() - Rejecting deviant pixels
[INFO]    :: combine.py 188 comb_frames() - Combining frames with a weightmean operation
[INFO]    :: combine.py 201 comb_frames() - Replacing completely masked pixels with the maxnonsat value of the input frames
[INFO]    :: combine.py 215 comb_frames() - 3 bias frames combined successfully!


In [11]:
tImage = traceimage.TraceImage(spectrograph, files=flat_files)

tflat = tImage.process(bias_subtract=bimage, trim=True)

[INFO]    :: processimages.py 412 bias_subtract() - Bias subtracting your image(s)
[INFO]    :: processimages.py 418 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: processimages.py 418 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: processimages.py 418 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: combine.py 62 comb_frames() - Combining 3 trace_image frames
[WORK IN ]::
[PROGRESS]:: combine.py 66 comb_frames() - lscomb feature has not been included here yet...
[INFO]    :: combine.py 95 comb_frames() - Finding saturated and non-linear pixels
[INFO]    :: combine.py 119 comb_frames() - Rejecting cosmic rays
[INFO]    :: combine.py 163 comb_frames() - Not rejecting any low/high pixels
[INFO]    :: combine.py 171 comb_frames() - Rejecting deviant pixels
[INFO]    :: combine.py 188 comb_frames() - Combining frames with a weightmean operation
[INFO]    :: combine.py 201 comb_frames() - Replacing completely masked pixels with

In [12]:
bpm = spectrograph.bpm(shape=tflat.shape, det=1)
ginga.show_image(tflat)

(<ginga.util.grc.RemoteClient at 0x7fbe3c8b7588>,
 <ginga.util.grc._channel_proxy at 0x7fbe3c8b7208>)

In [13]:
_, _ = ginga.show_image(tflat*(1.-bpm))